# High performance pandas

- När vi gör någon form utav mask


 ```py
 mask = (x > .5) & (y < .5 )
 df[df[mask]]
 ```


 under the hood 

```py
 tmp1 = (x > .5)
 tmp2 = (y < .5)
 mask = tmp1 & tmp2
 ```


# Vanlig eval är farlig när man använder den för user input, men pd är bra

 use pd.eval(""), df.query() -> does elementwise operation with numexpr
 
 # pipenv install numexpr,  for eval
 

In [2]:
import numpy as np
import pandas as pd

nrows, ncols = 1_000_000, 100

df1, df2, df3, df4 = [pd.DataFrame(np.random.randn(nrows, ncols)) for _ in range(4)]
df1.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-1.180668,-0.296170,2.125476,1.029353,1.007105,-0.178586,1.404289,-0.520112,1.593218,-0.567664,...,0.859018,-0.186204,-0.267388,0.804387,0.169641,1.092397,-0.566257,-0.186563,0.545230,0.169594
1,-0.016046,0.200632,-1.919886,-0.482389,-0.355757,-0.633904,-0.317077,-1.574686,-1.746732,0.008848,...,-0.407261,1.295177,-0.142328,0.303424,0.446034,-1.268404,-1.514920,1.362295,0.086117,-0.606083
2,0.812318,-0.026930,-0.274190,-1.376311,-0.376617,0.460141,-0.226272,0.236855,-1.528708,-0.696089,...,-2.122403,1.910891,0.757388,-0.964593,-0.095619,0.862726,-1.005201,-1.884780,-0.297415,0.594045
3,0.612275,0.812513,-1.054130,-1.419364,-0.464742,-0.466938,-0.423504,-0.883023,1.795970,1.785213,...,0.752832,0.635729,-1.045427,-0.285947,-0.849875,-1.015100,0.933666,-0.914767,1.025084,1.075309
4,-1.499901,-0.403597,-0.557945,-0.205185,-1.116373,-0.063119,2.316306,1.333205,0.018595,-0.123521,...,2.134965,2.321986,-0.234316,2.173443,0.272844,1.086552,-0.309052,0.640760,0.153969,-0.681684


In [4]:
df1.shape

(1000000, 100)

In [6]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 100 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   0       1000000 non-null  float64
 1   1       1000000 non-null  float64
 2   2       1000000 non-null  float64
 3   3       1000000 non-null  float64
 4   4       1000000 non-null  float64
 5   5       1000000 non-null  float64
 6   6       1000000 non-null  float64
 7   7       1000000 non-null  float64
 8   8       1000000 non-null  float64
 9   9       1000000 non-null  float64
 10  10      1000000 non-null  float64
 11  11      1000000 non-null  float64
 12  12      1000000 non-null  float64
 13  13      1000000 non-null  float64
 14  14      1000000 non-null  float64
 15  15      1000000 non-null  float64
 16  16      1000000 non-null  float64
 17  17      1000000 non-null  float64
 18  18      1000000 non-null  float64
 19  19      1000000 non-null  float64
 20  20      1000000 non-null

In [8]:
%timeit df1+df2+df3+df4
%timeit pd.eval("df1+df2+df3+df4")

757 ms ± 15.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
347 ms ± 3.59 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
standard = df1+df2+df3+df4
sum_eval = pd.eval("df1+df2+df3+df4")

sum_eval.equals(standard)


True

In [13]:
rolls = pd.DataFrame(np.random.randint(1,6, (6,3)), columns = ["Die1", "Die2", "Die3"])
rolls

,Die1,Die2,Die3
0,1,2,4
1,1,5,2
2,5,5,1
3,5,1,1
4,3,3,4
5,1,4,1


In [16]:
rolls.eval("Sum = Die1 + Die2 + Die3", inplace=True)
rolls

,Die1,Die2,Die3,Sum
0,1,2,4,7
1,1,5,2,8
2,5,5,1,11
3,5,1,1,7
4,3,3,4,10
5,1,4,1,6


In [19]:
# @ används för att man ska använda variabeln istället för columnen.

high = 11
rolls.eval("Winner = Sum > @high")

,Die1,Die2,Die3,Sum,Winner
0,1,2,4,7,False
1,1,5,2,8,False
2,5,5,1,11,False
3,5,1,1,7,False
4,3,3,4,10,False
5,1,4,1,6,False


In [23]:
# Traditional way to filter from a DF
rolls[rolls["Sum"] < high]

,Die1,Die2,Die3,Sum
0,1,2,4,7
1,1,5,2,8
3,5,1,1,7
4,3,3,4,10
5,1,4,1,6


## Query

- Filter using query

In [27]:
# Vill jag liknande som ovan ex
# Man tjänar tid med query på större dataset


rolls.query("Sum < @high & Die1 == 3")

,Die1,Die2,Die3,Sum
4,3,3,4,10


In [29]:
df_os = pd.read_csv("../Data/athlete_events.csv")

df_os.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [32]:
# Vi plockar ut bara sweden
# Traditionellt sätt att göra på
df_os[df_os["NOC"] == "SWE"].head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
725,414,Arvid berg,M,26.0,NaN,NaN,Sweden,SWE,1912 Summer,1912,Summer,Stockholm,Athletics,Athletics Men's Hammer Throw,NaN
726,415,Bjrn Olof Conny berg,M,23.0,181.0,76.0,Sweden,SWE,1992 Winter,1992,Winter,Albertville,Freestyle Skiing,Freestyle Skiing Men's Moguls,NaN
727,416,Nils Georg berg,M,19.0,181.0,78.0,Sweden,SWE,1912 Summer,1912,Summer,Stockholm,Athletics,Athletics Men's Long Jump,Bronze
728,416,Nils Georg berg,M,19.0,181.0,78.0,Sweden,SWE,1912 Summer,1912,Summer,Stockholm,Athletics,Athletics Men's Triple Jump,Silver
729,417,Sara Helena berg,F,17.0,190.0,73.0,Sweden,SWE,1988 Summer,1988,Summer,Seoul,Swimming,Swimming Women's 50 metres Freestyle,NaN


In [35]:
# Utnyttjar att man göra två olika typer av fnuttar eftersom vi söker efter en sträng 'SWE'
df_os.query("NOC == 'SWE'").head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
725,414,Arvid berg,M,26.0,NaN,NaN,Sweden,SWE,1912 Summer,1912,Summer,Stockholm,Athletics,Athletics Men's Hammer Throw,NaN
726,415,Bjrn Olof Conny berg,M,23.0,181.0,76.0,Sweden,SWE,1992 Winter,1992,Winter,Albertville,Freestyle Skiing,Freestyle Skiing Men's Moguls,NaN
727,416,Nils Georg berg,M,19.0,181.0,78.0,Sweden,SWE,1912 Summer,1912,Summer,Stockholm,Athletics,Athletics Men's Long Jump,Bronze
728,416,Nils Georg berg,M,19.0,181.0,78.0,Sweden,SWE,1912 Summer,1912,Summer,Stockholm,Athletics,Athletics Men's Triple Jump,Silver
729,417,Sara Helena berg,F,17.0,190.0,73.0,Sweden,SWE,1988 Summer,1988,Summer,Seoul,Swimming,Swimming Women's 50 metres Freestyle,NaN


In [36]:
%timeit df_os[df_os["NOC"] == "SWE"]
%timeit df_os.query("NOC == 'SWE'")

22.2 ms ± 1.86 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
11.5 ms ± 117 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [39]:
# Query behöver dock inte alltid vara snabbare
# den är lite snabbare när man maskar på flera villkor jämfört med traditionella sättet

%timeit df_os[df_os["Height"] > 180]
%timeit df_os.query("Height > 180")

10.9 ms ± 103 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
14.7 ms ± 427 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [41]:
df_os[(df_os["Sex"] == "F") & (df_os["Height"] > 180) & (df_os["NOC"] == "SWE")].head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
729,417,Sara Helena berg,F,17.0,190.0,73.0,Sweden,SWE,1988 Summer,1988,Summer,Seoul,Swimming,Swimming Women's 50 metres Freestyle,NaN
5175,2940,Jenny Alm,F,27.0,184.0,80.0,Sweden,SWE,2016 Summer,2016,Summer,Rio de Janeiro,Handball,Handball Women's Handball,NaN
7555,4210,Marina Vladimirovna Andrievskaia,F,29.0,182.0,66.0,Sweden,SWE,2004 Summer,2004,Summer,Athina,Badminton,Badminton Women's Singles,NaN
19070,10088,Anna Therese Bengtsson,F,29.0,187.0,83.0,Sweden,SWE,2008 Summer,2008,Summer,Beijing,Handball,Handball Women's Handball,NaN
28221,14643,Maria Helene Brandin,F,25.0,186.0,85.0,Sweden,SWE,1988 Summer,1988,Summer,Seoul,Rowing,Rowing Women's Double Sculls,NaN


In [42]:
df_os.query("Sex == 'F' & Height > 180 & NOC == 'SWE'").head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
729,417,Sara Helena berg,F,17.0,190.0,73.0,Sweden,SWE,1988 Summer,1988,Summer,Seoul,Swimming,Swimming Women's 50 metres Freestyle,NaN
5175,2940,Jenny Alm,F,27.0,184.0,80.0,Sweden,SWE,2016 Summer,2016,Summer,Rio de Janeiro,Handball,Handball Women's Handball,NaN
7555,4210,Marina Vladimirovna Andrievskaia,F,29.0,182.0,66.0,Sweden,SWE,2004 Summer,2004,Summer,Athina,Badminton,Badminton Women's Singles,NaN
19070,10088,Anna Therese Bengtsson,F,29.0,187.0,83.0,Sweden,SWE,2008 Summer,2008,Summer,Beijing,Handball,Handball Women's Handball,NaN
28221,14643,Maria Helene Brandin,F,25.0,186.0,85.0,Sweden,SWE,1988 Summer,1988,Summer,Seoul,Rowing,Rowing Women's Double Sculls,NaN


In [43]:
%timeit df_os[(df_os["Sex"] == "F") & (df_os["Height"] > 180) & (df_os["NOC"] == "SWE")]
%timeit df_os.query("Sex == 'F' & Height > 180 & NOC == 'SWE'")

34.8 ms ± 171 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
16.6 ms ± 460 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
